# **Básico** - *Carga de datos* 

### Carga de librerías y datos

In [5]:
#importamos librerías
import pandas as pd
import datetime as dt
import numpy as np

#cargamos los datos
df=pd.read_csv("OnlineRetail.csv", sep=";")

### Análisis exploratorio

In [6]:
#mostramos los primeros registros para ver las columnas y ejemplos de datos posibles
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [7]:
#vemos la información general del dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


Con lo anterior podemos saber que tiene 541909 registros y 8 columnas, en las cuales hay datos vacíos en varias de ellas, para saber la cantidad exacta de valores nulos en cada columna, podemos ejecutar la siguiente instrucción: 

In [8]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [9]:
#eliminamos los datos vacíos
df.dropna(inplace=True)

#comprobamos si ya no hay valores vacíos
df.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [10]:
#Obtenemos la media, máximo, mínimo y desviación estándar de las columnas con valores numéricos
df.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


Podemos observar que en la columna Quantity tenemos un valor mímino negativo, lo cual no puede ser posible considerando que no se pueden vender cantidades negativas de un artículo, por lo que vamos a eliminar los valores negativos en esa columna:

In [11]:
#definimos nuevamente el dataframe pero ahora solamente con las filas en donde Quantity tenga un valor mayor a 0
df=df[df.Quantity >= 0]

#calculamos nuevamente la media, máximo, mínimo y desviación estándar
df.describe()

,Quantity,UnitPrice,CustomerID
count,397924.000000,397924.000000,397924.000000
mean,13.021823,3.116174,15294.315171
std,180.420210,22.096788,1713.169877
min,1.000000,0.000000,12346.000000
25%,2.000000,1.250000,13969.000000
50%,6.000000,1.950000,15159.000000
75%,12.000000,3.750000,16795.000000
max,80995.000000,8142.750000,18287.000000


In [12]:
#calculamos cuántos vaores únicos podemos encontrar en cada una de las columnas
df.nunique()

InvoiceNo      18536
StockCode       3665
Description     3877
Quantity         302
InvoiceDate    17286
UnitPrice        441
CustomerID      4339
Country           37
dtype: int64

# **Intermedio** - *RFM* 

## Recency

In [19]:
#convertimos la columna InvoiceDate en un formato de fecha que podamos usar
df.loc[:,['InvoiceDate']] = pd.to_datetime(df['InvoiceDate'])


#obtenemos los datos agrupando por CustomerID y tomando los valores de las fechas
df_recency = df.groupby(by='CustomerID', as_index=False)['InvoiceDate'].max()

#definimos nombres de columnas
df_recency.columns=["CustomerID", "LastInvoice"]

#Creamos una variable para fechas recientes.
recent_date = df_recency['LastInvoice'].max()

#agregamos una columna para Recency y añadimos valores
df_recency['Recency'] = df_recency['LastInvoice'].apply(lambda x: (recent_date - x).days)

## Frequency

In [20]:
#definimos los IDs de los clientes eliminando duplicados y contamos las compras de acuerdo a las fechas
df_frequency = df.drop_duplicates().groupby(by=['CustomerID'], as_index=False)['InvoiceDate'].count()

#Nombramos columnas.
df_frequency.columns = ['CustomerId', 'Frequency'] 

## Monetary Value

In [21]:
#obtenemos el total por compra multiplicando el precio unitario * cantidad
df['Total'] = df['UnitPrice']*df['Quantity']

#obtenemos los datos agrupando por cliente
df_monetary = df.groupby(by='CustomerID', as_index=False)['Total'].sum()

#nombramos columnas
df_monetary.columns = ['CustomerID', 'Monetary']

## RFM completo

In [22]:
#creamos un nuevo dataframe agregando los datos obtenidos anteriormente de Recency, Frequency y Monetary Value
df_rfm = pd.DataFrame([df_recency.CustomerID,df_recency.Recency,df_frequency.Frequency,df_monetary.Monetary]).transpose()

#mostramos el dataframe
df_rfm.head()

,CustomerID,Recency,Frequency,Monetary
0,12346.0,325.0,1.0,77183.60
1,12347.0,1.0,182.0,4310.00
2,12348.0,74.0,31.0,1797.24
3,12349.0,18.0,73.0,1757.55
4,12350.0,309.0,17.0,334.40


# Asignación de scores
(Nota: se tomará como la mejor puntuación al número 1, es decir, los clientes con esta puntuación son considerados los más frecuentes y con mayor valor monetario)



In [31]:
#definimos los cuantiles, en este caso, los puntajes van de 1 a 5, por lo que los porcentajes serán de 20
quantiles = df_rfm.quantile(q=[0.2,0.4,0.6, 0.8])
quantiles = quantiles.to_dict()

#creamos un nuevo df a partir del df_rfm
rfm_nuevo = df_rfm

#creamos una función para que retorne el score de acuerdo a las cantidades y porcentajes de los cuantiles
#siendo 1 la mejor puntuación, considerado para las compras más recientes
def RScore(x,p,d):
    if x <= d[p][0.2]:
        return 1
    elif x <= d[p][0.4]:
        return 2
    elif x <= d[p][0.6]: 
        return 3
    elif x <= d[p][0.8]: 
        return 4
    else:
        return 5

#creamos otra función similar pero ahora para los valores de frequency y monetary
#siendo 1 la mejor puntuación, considerado para los clientes con compras más frecuentes y mayor valor monetario
def FMScore(x,p,d):
    if x <= d[p][0.2]:
        return 5 
    elif x <= d[p][0.4]:
        return 4
    elif x <= d[p][0.6]: 
        return 3
    elif x <= d[p][0.8]: 
        return 2
    else:
        return 1

#agregamos las columnas con los score calculados con las funciones creadas anteriormente
rfm_nuevo['r_quartile'] = rfm_nuevo['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_nuevo['f_quartile'] = rfm_nuevo['Frequency'].apply(FMScore, args=('Frequency',quantiles,)) 
rfm_nuevo['m_quartile'] = rfm_nuevo['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

#mostramos
rfm_nuevo.head()

,CustomerID,Recency,Frequency,Monetary,r_quartile,f_quartile,m_quartile
0,12346.0,325.0,1.0,77183.60,5,5,1
1,12347.0,1.0,182.0,4310.00,1,1,1
2,12348.0,74.0,31.0,1797.24,4,3,2
3,12349.0,18.0,73.0,1757.55,2,2,2
4,12350.0,309.0,17.0,334.40,5,4,4
